In [1]:
## Interpolarion simplificaton

In [2]:
import math


## Interpolarion throttle - gripper angle



l1 = 0.25 # m
l2 = 0.464 # m
dorso_center = 0.152 # m 
dist_robot = 0.51 # m
scotter_dic = 0.1 # m
scotter_mid = 0.3 # 
dorso_dic = scotter_mid - dorso_center - 0.04 # 102 mm



def gripper(angle_th):


    gripperangle = angle_th * 18 # considering that the gripper just moves between 0 and 180 


    d_th = abs(math.sin(gripperangle))

    #dR =  angle_st * 0.001 + d_th * 0.01

    #dL = - angle_st * 0.001 + d_th * 0.01
    


    return gripperangle



def calculate_IK( angle_th, l1, l2, dorso_dic, dist_robot):



    gripper_angle = gripper(angle_th)


    return gripper_angle 






angles_resultR_gripper = []
angles_resultL_gripper = []
angles_th = []


# Min: 0 deg, Max, 10 deg, Step 2 deg)

for j in range(0,12,2):
    x = calculate_IK(j, l1, l2, dorso_dic, dist_robot)
    angles_th.append(j)
    angles_resultR_gripper.append(x)
    angles_resultL_gripper.append(x)

print(angles_resultR_gripper)





[0, 36, 72, 108, 144, 180]


In [3]:

## Interpolarion throttle - gripper angle


#Step 1 
#Find angle between the dorso and arm 

l1 = 0.25 # m
l2 = 0.464 # m
dorso_center = 0.152 # m 
dist_robot = 0.51 # m
scotter_dic = 0.1 # m
scotter_mid = 0.3 # 
dorso_dic = scotter_mid - dorso_center - 0.04 # 102 mm



def angle_dorso_arm(dorso_dic, l1, l2):

    l = l1 + l2

    theta = math.asin(dorso_dic/l)
    
    return theta


def  angle_move_goal(tx, ty, l1, l2):

        # 1. Calculate l3
    l3 = math.sqrt( tx**2 + ty**2 )

    # 2. Calculate α_2 knee angle
    a2 = math.acos( (l1**2 + l2**2 - l3**2)/(2*l1*l2) )


    # 3. Calculate theta_2
    theta2d = math.pi - a2

    # 4. Calculate α_1
    # Sine law: math.sin(a2)/l3 = math.sin(a1)/l2

    a1 = math.asin( math.sin(a2)* l2/l3 )
    alpha = math.atan2( ty, tx )

    # Calculate theta1 from a2 and beta
    theta1 = alpha - a1
    
    return a2, theta1


def gripper(angle_th, angle_st):


    gripperangle = angle_th * 18 # considering that the gripper just moves between 0 and 180 


    d_th = abs(math.sin(gripperangle))

    dR =  angle_st * 0.001 + d_th * 0.01

    dL = - angle_st * 0.001 + d_th * 0.01
    


    return dL, dR, gripperangle



def calculate_IK(angle_st, angle_th, l1, l2, dorso_dic, dist_robot):

    angles_right = []

    angles_left = [] 


    dL, dR, gripper_angle = gripper(angle_th, angle_st)


    Dist = [dL, dR]

    theta1R = angle_dorso_arm(dorso_dic, l1, l2)

    theta1L = theta1R # shoulder 



    dist_robotL = dist_robot + dL
    

    dist_robotR = dist_robot + dR 

    alphaR, theta2R = angle_move_goal(dorso_dic, dist_robotR,l1, l2)

    alphaL, theta2L = angle_move_goal(dorso_dic, dist_robotL, l1, l2)


    # append the angles 

    # shoulder Lateral

    angles_right.append(theta1R/math.pi * 180.0)

    angles_left.append(theta1L/math.pi * 180.0)

    #should frontal

    angles_right.append(theta2R/math.pi * 180.0)

    angles_left.append(theta2L/math.pi * 180.0)

    # elbow

    angles_right.append(alphaR/math.pi * 180.0)

    angles_left.append(alphaL/math.pi * 180.0)


    return angles_right, angles_left


# For simplifications reason it was fixed throttle angle, otherwise the problem will be complex


angles_resultR_SE = []
angles_resultL_SE = []
angles_st = []

for i in range(-20,21, 5):
    j = 5
    angles_st.append(i)
    xR, xL = calculate_IK(i, j, l1, l2, dorso_dic, dist_robot)
        
    angles_resultR_SE.append(xR)
    angles_resultL_SE.append(xL)







In [5]:
import numpy as np


final_angles = []

def interpolation_angles(new_ste, new_th, angles_th, angles_st, angles_resultR_gripper, angles_resultR_SE):

    gripper_new = np.interp(new_th,angles_th,angles_resultR_gripper)

    final_angles.append(gripper_new)

    x = angles_st

    y = angles_resultR_SE

    y_new = []

    y_new2 = []

    y_new3 = []

    final_new_angle = [] 

    for i in range(len(x)):

        y_new.append(y[i][0])

    for i in range(len(x)):

        y_new2.append(y[i][1])

    for i in range(len(x)):

        y_new3.append(y[i][2])

    f1 = np.interp(new_ste,x,y_new)

    final_angles.append(f1)

    f2 = np.interp(new_ste,x,y_new2)

    final_angles.append(f2)

    f3 = np.interp(new_ste,x,y_new3)

    final_angles.append(f3)

    return final_angles


new_ste = -15
new_th = 1
final_angles = interpolation_angles(new_ste, new_th, angles_th, angles_st, angles_resultR_gripper, angles_resultR_SE)

print(final_angles)


[18.0, 8.699981681146145, 13.868969985093532, 86.99145113542805]
